In [1]:
import torch
import torch.nn as nn
import numpy as np
import math
from spyrit.core.Forward_Operator import *
from spyrit.core.Data_Consistency import *

## Instantiate Forward_Operator

In [2]:
img_size = 32*32
nb_measurements = 400
batch_size = 10
Hcomplete = walsh_matrix(img_size)
Hsub = Hcomplete[0:nb_measurements,:]
FO = Forward_operator(Hsub)

## Pinv_orthogonal

In [3]:
Pinv_ortho = Pinv_orthogonal()

### forward

In [4]:
x = torch.tensor(np.random.random([batch_size,nb_measurements]), dtype=torch.float)
y = Pinv_ortho(x, FO)
print(y.shape)

torch.Size([10, 1024])


## learned_measurement_to_image

In [5]:
Meas_to_Img = learned_measurement_to_image(32*32, 400)

In [6]:
y = Meas_to_Img(x, FO)
print(y.shape)

torch.Size([10, 1024])


## gradient_step

In [11]:
GS = gradient_step()

### forward

In [ ]:
x = torch.tensor(np.random.random([10,400]), dtype=torch.float) 
x_0 = torch.tensor(np.random.random([10,32*32]), dtype=torch.float) 